# Automatic tagging of StackOverflow dataset

Meyi Duleme, OpenClassrooms/Centrale Supélec


## Load cleaned data

In [1]:
import pickle
import numpy as np
a_file = open("tfidf_vocab.pkl", "rb")
tfidf_vocab = pickle.load(a_file) 

In [2]:
a_file = open("bow_vocab.pkl", "rb")
bow_vocab = pickle.load(a_file)

In [3]:
a_file = open("X_bagofwords_train.pkl", "rb")
X_bagofwords_train = pickle.load(a_file)

In [4]:
open_file = open("X_bagofwords_test.pkl", "rb")
X_bagofwords_test = pickle.load(open_file)
open_file.close()

In [5]:
open_file = open("X_tfidf_train.pkl", "rb")
X_tfidf_train = pickle.load(open_file)
open_file.close()

In [6]:
open_file = open("X_tfidf_test.pkl", "rb")
X_tfidf_test = pickle.load(open_file)
open_file.close()

In [7]:
open_file = open("tokens.pkl", "rb")
tokens = pickle.load(open_file)
open_file.close()

In [8]:
open_file = open("y_train.pkl", "rb")
y_train = np.array(pickle.load(open_file))
open_file.close()

In [9]:
open_file = open("y_test.pkl", "rb")
y_test = np.array(pickle.load(open_file))
open_file.close()

##  Latent Dirichlet Allocation LDA

In [10]:
from gensim import corpora, models


list_of_list_of_tokens = tokens
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

C:\Users\dulem\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Wall time: 43.1 s


In [11]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.049*"class" + 0.019*"x" + 0.019*"foo" + 0.015*"static" + 0.013*"use" + 0.012*"difference" + 0.011*"type" + 0.011*"def" + 0.011*"interface" + 0.011*"variable"

1: 0.288*"'" + 0.041*"`" + 0.034*"=" + 0.020*"table" + 0.020*"``" + 0.017*"key" + 0.016*"''" + 0.015*"column" + 0.015*"name" + 0.011*"$"

2: 0.029*"page" + 0.026*"text" + 0.019*"like" + 0.018*"want" + 0.017*"using" + 0.015*"html" + 0.015*"n't" + 0.015*"javascript" + 0.013*"way" + 0.013*"element"

3: 0.145*"[" + 0.145*"]" + 0.056*"=" + 0.047*"1" + 0.032*"2" + 0.031*"0" + 0.026*"%" + 0.024*"3" + 0.020*"-" + 0.018*"*"

4: 0.018*"app" + 0.013*"application" + 0.013*"using" + 0.012*"android" + 0.012*"n't" + 0.011*"'s" + 0.010*"'m" + 0.009*"-" + 0.009*"use" + 0.009*"project"

5: 0.033*"data" + 0.026*"user" + 0.017*"server" + 0.012*"database" + 0.010*"application" + 0.010*"would" + 0.010*"client" + 0.010*"web" + 0.010*"using" + 0.009*"service"

6: 0.034*"list" + 0.031*"image" + 0.028*"number" + 0.019*"way" + 0.017*"string" + 0.016*"

In [12]:
lda_model[corpus[0]] 

[(8, 0.07739457), (9, 0.076798305), (14, 0.73647875), (17, 0.098048255)]

In [13]:
from functions import *
new_text = 'I love eating python at every meal.'
_, tokens  = text_prepare(new_text)
lda_model[dictionary_LDA.doc2bow(tokens)]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dulem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[(4, 0.31753632), (8, 0.23809375), (14, 0.4036289)]

In [14]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\dulem\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


## Logistic regression

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

C:\Users\dulem\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
X_tfidf_train.shape

C:\Users\dulem\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(16617, 10774)

In [17]:
y_train.shape

C:\Users\dulem\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(16617, 50)

In [18]:
model_logistic = OneVsRestClassifier(LogisticRegression(penalty='l2'), n_jobs=-1)
model_logistic.fit(X_tfidf_train, y_train)
predicted_logistic = model_logistic.predict(X_tfidf_test)

C:\Users\dulem\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_logistic))

              precision    recall  f1-score   support

           0       0.84      0.49      0.62       750
           1       0.95      0.62      0.75       800
           2       0.83      0.43      0.57       598
           3       0.97      0.59      0.73       475
           4       0.97      0.79      0.87       377
           5       0.77      0.15      0.25       381
           6       0.92      0.25      0.40       326
           7       0.60      0.19      0.29       290
           8       0.79      0.28      0.42       260
           9       0.90      0.52      0.66       220
          10       0.89      0.40      0.55       231
          11       0.83      0.27      0.41       196
          12       0.62      0.03      0.05       198
          13       1.00      0.12      0.21       187
          14       0.66      0.22      0.33       149
          15       0.94      0.50      0.65       146
          16       0.73      0.07      0.14       147
          17       0.72    

C:\Users\dulem\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\dulem\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dulem\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dul

In [20]:
import joblib
joblib.dump(model_logistic, 'lr.joblib') 

C:\Users\dulem\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['lr.joblib']